문제 1: 채널별 고객 획득 비용(CAC) 및 생애 가치(LTV) 분석
목표: 마케팅 채널의 진정한 ROI를 평가하기 위해 CAC와 LTV를 계산하고 최적 투자 전략을 제안하세요.
1. 데이터 로드 및 전처리
2. 채널별 CAC 계산
    - 2024년 데이터만 사용 - (2024년 마케팅데이터만 사용 -> `24년 채널별 마케팅비용 / 24년 신규 획득 고객수`)
    - 각 채널별 총 마케팅 비용 집계
    - 각 채널별 신규 고객 수 집계 (2024년 첫 거래 기준)
    - CAC = 총 마케팅 비용 / 신규 고객 수
    - organic과 referral 채널은 CAC = 0으로 처리
3. 고객별 LTV 계산
4. 채널별 LTV 및 ROI 분석 (`24년도 신규 고객들의 평균 LTV / 24년 CAC`)
    - 채널별 평균 LTV 계산
    - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
    - payback period = CAC / (월평균 구매금액) 계산
5. 시각화 및 전략 제안 (15분)
    - CAC vs LTV 산점도 차트
    - 채널별 ROI 막대 차트
    - 마케팅 예산 재배분 제안 (현재 vs 최적 배분)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# csv파일 불러오기
satis_df = pd.read_csv('./customer_satisfaction.csv', encoding='EUC-KR')
trans_df = pd.read_csv('./customer_transactions.csv', encoding='EUC-KR')
perf_df = pd.read_csv('./marketing_performance.csv', encoding='EUC-KR')

# 2. 날짜 처리
trans_df["transaction_date"] = pd.to_datetime(trans_df["transaction_date"])
trans_df["registration_date"] = pd.to_datetime(trans_df["registration_date"])
perf_df["month"] = pd.to_datetime(perf_df["month"])

# 3. 2024년 CAC 계산용 마케팅 데이터 필터링
perf_2024 = perf_df[perf_df["month"].dt.year == 2024]
cac_df = perf_2024.groupby("channel").agg({
    "marketing_spend": "sum",
    "new_customers": "sum"
}).reset_index()

# CAC 계산 (organic, referral은 0으로 설정)
cac_df["cac"] = cac_df.apply(
    lambda row: 0 if row["channel"] in ["organic", "referral"] else row["marketing_spend"] / row["new_customers"],
    axis=1
)

# 4. 2024년에 첫 거래한 고객만 추출
first_purchase = trans_df.sort_values("transaction_date").groupby("customer_id").first().reset_index()
new_customers_2024 = first_purchase[first_purchase["transaction_date"].dt.year == 2024]
customer_channel_map = new_customers_2024[["customer_id", "acquisition_channel"]]

# 5. 2024년 전체 거래 데이터에 채널 정보 병합
trans_2024 = trans_df[trans_df["transaction_date"].dt.year == 2024]
trans_2024 = trans_2024.merge(customer_channel_map, on="customer_id", how="inner")

# 6. 채널별 LTV 계산
ltv_df = trans_2024.groupby("acquisition_channel")["order_value"].agg(["sum", "count"]).reset_index()
ltv_df.columns = ["channel", "total_order_value", "num_orders"]
ltv_df["customers"] = trans_2024.groupby("acquisition_channel")["customer_id"].nunique().values
ltv_df["avg_ltv"] = ltv_df["total_order_value"] / ltv_df["customers"]

# 7. CAC + LTV + ROI 병합
summary_df = pd.merge(cac_df, ltv_df, on="channel", how="left")
summary_df["roi"] = summary_df.apply(lambda row: np.inf if row["cac"] == 0 else row["avg_ltv"] / row["cac"], axis=1)
summary_df["monthly_spend"] = summary_df["avg_ltv"] / 12
summary_df["payback_period"] = summary_df.apply(
    lambda row: np.nan if row["cac"] == 0 else row["cac"] / row["monthly_spend"], axis=1
)

# 정리
summary_df = summary_df[["channel", "cac", "avg_ltv", "roi", "payback_period"]]
print(summary_df)

# 8. 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# CAC vs LTV 산점도
axes[0].scatter(summary_df["cac"], summary_df["avg_ltv"], color="skyblue")
for i, row in summary_df.iterrows():
    axes[0].text(row["cac"], row["avg_ltv"], row["channel"], fontsize=9)
axes[0].set_xlabel("CAC")
axes[0].set_ylabel("LTV")
axes[0].set_title("CAC vs LTV")

# ROI 막대그래프
axes[1].bar(summary_df["channel"], summary_df["roi"], color="lightgreen")
axes[1].set_ylabel("ROI (LTV / CAC)")
axes[1].set_title("Channel-wise ROI")

plt.tight_layout()
plt.show()

KeyError: 'acquisition_channel'

In [5]:
perf_df['channel'].values

array(['organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'paid_search', 'social_media', 'email', 'referral',
       'organic', 'p

문제 2: 고객 코호트 분석 및 유지율 개선 전략
목표: "월별 가입 코호트"의 유지율과 매출 기여도를 분석하여 고객 유지 전략을 수립하세요.
1. 코호트 그룹 생성
2. 월별 유지율 계산
    - 각 거래의 코호트 대비 경과 개월 수 계산
    - 코호트별 Month 0, 1, 2, ... 12에서의 활성 고객 수
    - 유지율 테이블 생성 (코호트 × 경과월)
    - 평균 유지율 곡선 계산
3. 코호트별 매출 기여도 분석
    - 각 코호트의 월별 누적 매출 계산
    - 코호트별 고객 1인당 누적 매출 (ARPU) 계산
    - 6개월 후, 12개월 후 예상 LTV 추정
4. 유지율 히트맵 시각화
5. 개선 전략 수립
    - 유지율이 급격히 감소하는 시점 식별
    - 고성과 코호트 vs 저성과 코호트 특성 비교
    - 유지율 개선을 위한 액션 아이템 3가지 제안

In [ ]:
# 1. 데이터 불러오기
df = pd.read_csv("customer_transactions.csv")
df["registration_date"] = pd.to_datetime(df["registration_date"])
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

# 2. 코호트 및 월별 전처리
df["cohort"] = df["registration_date"].dt.to_period("M")
df["tx_month"] = df["transaction_date"].dt.to_period("M")

# 경과 개월 수 계산 함수
def get_month_diff(start, end):
    return (end.year - start.year) * 12 + (end.month - start.month)

df["cohort_index"] = df.apply(
    lambda row: get_month_diff(row["cohort"].to_timestamp(), row["tx_month"].to_timestamp()), axis=1
)

# 3. 유지율 테이블 생성
cohort_counts = df.groupby(["cohort", "cohort_index"])["customer_id"].nunique().unstack(1)
cohort_size = cohort_counts.iloc[:, 0]
retention = cohort_counts.divide(cohort_size, axis=0)

# 4. 평균 유지율 곡선
avg_retention = retention.mean()

# 5. 매출 테이블 및 ARPU 계산
revenue = df.groupby(["cohort", "cohort_index"])["order_value"].sum().unstack(1)
cumulative_revenue = revenue.cumsum(axis=1)
arpu = cumulative_revenue.divide(cohort_size, axis=0)

# 6. 시각화
plt.figure(figsize=(12, 5))
sns.heatmap(retention, annot=True, fmt=".0%", cmap="YlGnBu")
plt.title("Retention Rate by Cohort")
plt.ylabel("Cohort (Registration Month)")
plt.xlabel("Months Since Registration")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(avg_retention.index, avg_retention.values, marker="o")
plt.title("Average Retention Curve")
plt.xlabel("Months Since Registration")
plt.ylabel("Average Retention Rate")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(arpu.mean().index, arpu.mean().values, marker="o", color='orange')
plt.title("Average Revenue per User (ARPU) Over Time")
plt.xlabel("Months Since Registration")
plt.ylabel("Average Revenue ($)")
plt.grid(True)
plt.tight_layout()
plt.show()

# 7. LTV 요약
ltv_6 = arpu[6].mean()
ltv_12 = arpu[12].mean()

print("📌 LTV Summary:")
print(f" - 6개월 후 LTV 평균: ${ltv_6:.2f}")
print(f" - 12개월 후 LTV 평균: ${ltv_12:.2f}")